In [1]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", sep='delimiter',header=None) #, names=["Players", "Outcome"]
df.head()

<ipython-input-14-db80963f8dd2>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", sep='delimiter',header=None) #, names=["Players", "Outcome"]


,0
0,"Shafiul to Harris, 2 runs"
1,"Shafiul to Hussey, 1 wide, OUT"
2,"Shafiul to Hussey, 2 runs"
3,"Shafiul to Smith, 1 run"
4,"Shafiul to Hussey, 1 run"


In [ ]:
# Bowler, batsman 
# Runs in numbers/words, out else 0. If wide/no ball then ignore entry
# out